In [71]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import hashlib

In [64]:
categories = pd.read_csv("./data/categories.csv")
customers = pd.read_csv("./data/customers.csv")
employees = pd.read_csv("./data/employees.csv")
orders = pd.read_csv("./data/orders.csv")
products = pd.read_csv("./data/products.csv")
suppliers = pd.read_csv("./data/suppliers.csv")

In [3]:
categories.head()

,CategoryID,CategoryName,Description,Picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",\x
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",\x
2,3,Confections,"Desserts, candies, and sweet breads",\x
3,4,Dairy Products,Cheeses,\x
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",\x


In [4]:
customers.head()

,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NaN,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,NaN,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,NaN,05023,Mexico,(5) 555-3932,NaN
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,NaN,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,NaN,S-958 22,Sweden,0921-12 34 65,0921-12 34 67


In [5]:
employees.head()

,EmployeeID,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Photo,Notes,ReportsTo,PhotoPath
0,1,Davolio,Nancy,Sales Representative,Ms.,1948-12-08,1992-05-01,507 - 20th Ave. E.\nApt. 2A,Seattle,WA,98122,USA,(206) 555-9857,5467,\x,Education includes a BA in psychology from Col...,2.0,http://accweb/emmployees/davolio.bmp
1,2,Fuller,Andrew,"Vice President, Sales",Dr.,1952-02-19,1992-08-14,908 W. Capital Way,Tacoma,WA,98401,USA,(206) 555-9482,3457,\x,Andrew received his BTS commercial in 1974 and...,NaN,http://accweb/emmployees/fuller.bmp
2,3,Leverling,Janet,Sales Representative,Ms.,1963-08-30,1992-04-01,722 Moss Bay Blvd.,Kirkland,WA,98033,USA,(206) 555-3412,3355,\x,Janet has a BS degree in chemistry from Boston...,2.0,http://accweb/emmployees/leverling.bmp
3,4,Peacock,Margaret,Sales Representative,Mrs.,1937-09-19,1993-05-03,4110 Old Redmond Rd.,Redmond,WA,98052,USA,(206) 555-8122,5176,\x,Margaret holds a BA in English literature from...,2.0,http://accweb/emmployees/peacock.bmp
4,5,Buchanan,Steven,Sales Manager,Mr.,1955-03-04,1993-10-17,14 Garrett Hill,London,NaN,SW1 8JR,UK,(71) 555-4848,3453,\x,Steven Buchanan graduated from St. Andrews Uni...,2.0,http://accweb/emmployees/buchanan.bmp


In [6]:
orders.head()

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,OrderID.1,ProductID,UnitPrice,Quantity,Discount
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,NaN,51100,France,10248,11,14.0,12,0.0
1,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,NaN,51100,France,10248,42,9.8,10,0.0
2,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,NaN,51100,France,10248,72,34.8,5,0.0
3,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,NaN,44087,Germany,10249,14,18.6,9,0.0
4,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,NaN,44087,Germany,10249,51,42.4,40,0.0


In [7]:
products.head()

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,8,1,10 boxes x 30 bags,18.00,39,0,10,1
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,1
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


In [8]:
suppliers.head()

,SupplierID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,NaN,EC1 4SD,UK,(171) 555-2222,NaN,NaN
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,NaN,#CAJUN.HTM#
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,MI,48104,USA,(313) 555-5735,(313) 555-3349,NaN
3,4,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai Musashino-shi,Tokyo,NaN,100,Japan,(03) 3555-5011,NaN,NaN
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Export Administrator,Calle del Rosal 4,Oviedo,Asturias,33007,Spain,(98) 598 76 54,NaN,NaN


## Create Primary Key

In [107]:
def create_hash_key(input_value):
    input_value = str(input_value)
    hasher = hashlib.sha256()
    hasher.update(input_value.encode())
    hash_number = int(hasher.hexdigest(), 16)
    positive_hash_int = hash_number % (100000)

    return positive_hash_int

def create_primary_key(dataframe, input_columns):
    pk_column = None

    if len(input_columns) == 1:
        pk_column = dataframe[input_columns[0]].apply(create_hash_key)
    else:
        combined_column = dataframe[input_columns].apply(lambda row: "_".join(row.values.astype(str)), axis=1)
        pk_column = combined_column.apply(create_hash_key)

    return pk_column

In [108]:
input_columns = ["CategoryID"]
categories["CategoryID_"] = create_primary_key(categories, input_columns=input_columns)

input_columns = ["CompanyName"]
suppliers["SupplierID_"] = create_primary_key(suppliers, input_columns=input_columns)

input_columns = ["ProductName"]
products["ProductID_"] = create_primary_key(products, input_columns=input_columns)

input_columns = ["OrderDate", "RequiredDate", "ShippedDate"]
orders["OrderID_"] = create_primary_key(orders, input_columns=input_columns)


## Create nodes

In [124]:
def create_session(uri, user, password):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    return driver.session()

def create_nodes(session, node_label, pk_property_name, pk_pro_val, properties):

    for property_ in properties:
        primary_key_value = property_.pop(pk_pro_val)
        cypher = """
            MERGE (n: {0} {{{1}: $pk_pro}})
            SET n += $property_
        """.format(node_label, pk_property_name)

        session.run(cypher, pk_pro=primary_key_value, property_=property_)

In [126]:
uri = "neo4j://localhost:7687"
user = "neo4j"
password = None

session = create_session(uri, user, password)

create_nodes(session=session, node_label="Suppliers", 
            pk_property_name="supplierID_", pk_pro_val="SupplierID_", 
            properties=suppliers.to_dict(orient="records"))

create_nodes(session=session, node_label="Products",
            pk_property_name="productID_", pk_pro_val="ProductID_",
            properties=products.to_dict(orient="records"))

In [136]:
def create_relationship(session, node_a, key_a, node_b, key_b, related):
    cypher = """
        MATCH (a:{0})
        MATCH (b:{2})
        WHERE a.`{1}` = b.`{3}`
        MERGE (a)-[:{4}]->(b)
    """.format(node_a, key_a, node_b, key_b, related)

    session.run(cypher)

In [137]:
create_relationship(session=session, 
                    node_a="Suppliers", key_a="SupplierID",
                    node_b="Products", key_b="ProductID",
                    related="SUPPLIED"
                    )